To run plotly/widgets in jupyterlab:
``` shell
pip install jupyterlab "ipywidgets>=7.5"
curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
sudo apt-get install -y nodejs
jupyter labextension install jupyterlab-plotly@4.14.3
```

# Unavailable Space testing Testing

### Imports

In [1]:
import plotly.graph_objects as go
import numpy as np

from willitfit.plotting import plotter, plot_helper
from willitfit.params import VOL_UNAVAILABLE, VOL_EMPTY

## Helper Functions

In [2]:
def generate_wireframe(coords_arr):
    '''Quickly generate wireframe from coords arr'''
    meshes = []
    line_marker = dict(color='#0066FF', width=2)
    for arr in coords_arr:
        x,y,z = arr
        meshes.append(go.Scatter3d(x=x,
                                y=z,
                                z=y,
                                mode='lines',
                                #color='grey',
                                line=line_marker))

    return meshes

In [154]:
def get_volume_space(car_model, data, ratio_height=0.5, ratio_depth=0.5, rate_of_slant=2):
    """
    Returns available volume for a specific car_model.
    Use ratio_door to tweak estimated unavailable space caused by 45-degree trunk-door slope.
    """
    # Isolate dimension cols
    dim_cols = ['depth', 'height', 'width']

    # Trunk dimensions
    model_row = data[data['car_model'] == car_model]
    trunk_dims = model_row[dim_cols].to_numpy(int)[0]

    # Cuboid Volume Space
    volume_space = np.full(trunk_dims, VOL_EMPTY, dtype=int)

    # Unavailable space
    height_block = int(trunk_dims[1]*ratio_height)
    depth_block = int(trunk_dims[0]*ratio_depth)
    for i in range(1, depth_block):
        volume_space[i,-height_block:int(rate_of_slant)*i,:] = VOL_UNAVAILABLE

    return volume_space


In [138]:
def test_unavailable_space(car_model, data, ratio_height, ratio_depth, rate_of_slant):
    test_vol = get_volume_space(car_model=car_model, data=data, ratio_height=ratio_height, ratio_depth=ratio_depth, rate_of_slant=rate_of_slant)
    unavail_coords = np.argwhere(test_vol == VOL_UNAVAILABLE).T
    wireframe = generate_wireframe([unavail_coords])

    return plotter.draw_3d_plot(wireframe, test_vol.shape)

# Test

In [107]:
import pandas as pd
import os
from pathlib import Path

In [108]:
CSV_PATH = Path(os.path.abspath(os.getcwd())).parent.absolute()/"raw_data/cars.csv"
print(CSV_PATH)

/Users/tzu-fantang/code/proxvision/freichel/willitfit/raw_data/cars.csv


In [109]:
data = pd.read_csv(CSV_PATH)
car_model = data.iloc[250]['car_model']

In [110]:
data.iloc[250]

make                                                BMW
car_model    BMW 7 Active Hybrid 4.4l 4dr saloon 2010\n
depth                                               109
height                                               52
width                                                50
Name: 250, dtype: object

In [134]:
get_volume_space(car_model, data)

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0

In [155]:
test_unavailable_space(car_model=car_model, data=data, ratio_height=0.5, ratio_depth=0.5, rate_of_slant=2)